# Channel Polarization

Consider 2-bit transmitting channel such that:
- one bit is transmitted without any process
- another bit is transmitted with <b>XOR</b> operation of two inputs of bits.

We call these channels are <i>polarized</i> as it transmits the single bit <b>twice</b>. Thus, the twice-transmitted bit is <i>repetively coded</i>, therefore, it is a good channel. Otherwise, it is a bad channel.
We may demonstrate such channel using this $G_{2}$ matrix.

$$G_{2} = \begin{bmatrix}1 & 0\\
1 & 1
\end{bmatrix}$$

Consider inputs $x = \begin{bmatrix}u_{0} & u_{1}\end{bmatrix}$.

Transmitting $y=x G_{2}$ gives the same as we discussed. See the example below.

In [6]:
import numpy as np

x1 = np.array([0, 0])
x2 = np.array([0, 1])
x3 = np.array([1, 0])
x4 = np.array([1, 1])

G2 = np.array([[1, 0],[1, 1]])

print("G2:")
print(G2)

print("x1G2:")
print(np.matmul(x1,G2))
print("x2G2:")
print(np.matmul(x2,G2))
print("x3G2:")
print(np.matmul(x3,G2))
print("x4G2:")
print(np.matmul(x4,G2))

G2:
[[1 0]
 [1 1]]
x1G2:
[0 0]
x2G2:
[1 1]
x3G2:
[1 0]
x4G2:
[2 1]


In addition, we use 1 or 0 (bits) for the data. XOR operation for $x_{4}G_{2}$ was needed. The XOR operation can be implemented by adding the bits and then the (mod 2) operation. Adding in mod 2 is the addition in Galois field of $GF(2)$.

In [8]:
print("mod 2 of x4G2:")
print(np.matmul(x4,G2)%2)

mod 2 of x4G2:
[0 1]


Thus, this $G_{2}$ matrix is a one-to-one mapping for Galois field of $GF(2^{2})$. Channel polarization is a linear-based process, thus, it is faster than the other complicated encoders. See the example below.

In [11]:
X = np.array([[0, 0],
             [0, 1],
             [1, 0],
             [1, 1]])

Y = np.matmul(X,G2)%2

print("X:")
print(X)
print("Y:")
print(Y)

X:
[[0 0]
 [0 1]
 [1 0]
 [1 1]]
Y:
[[0 0]
 [1 1]
 [1 0]
 [0 1]]


One may notice that the $u_{0}$ is the bad channel, and the $u_{1}$ is the good channel.

One may <b>not</b> use $u_{0}$ for the data transmission, but as a redundancy. (<i>i.e.</i> We call this as <i><b>Frozen Bit</b></i>.)

If we only use $u_{1}$ for the data transmission, and we leave $u_{0} = 0$ as the frozen bit, we get:

In [16]:
u0 = X[0:2,0].reshape(2,1)
u1 = X[0:2,1].reshape(2,1)
print("u0:")
print(u0)
print("u1:")
print(u1)

print("Y:")
print(Y[0:2])

u0:
[[0]
 [0]]
u1:
[[0]
 [1]]
Y:
[[0 0]
 [1 1]]


We can clearly see that the channel polarization is based on the repetive coding.

It gets more complicated when we transmit more than 2 bits. The solution is "<b>Recursive</b>." If we recursively polarize the channels, let's say, twice, then we transmit 4 bits.

Polarizing function was solely demonstrated by $G_{2}$. How can we build $G_{4}$? <b>Recursively</b>. But how recursive?

Let's say we have $x=\begin{bmatrix}u_{0} & u_{1} & u_{2} & u_{3}\end{bmatrix}$. Can we polarize for $\begin{bmatrix}w_{0} & w_{2}\end{bmatrix}$ and $\begin{bmatrix}w_{1} & w_{3}\end{bmatrix}$ after polarizing $\{w\}$ by multiplying $G_{2}$ for $\begin{bmatrix}u_{0} & u_{1}\end{bmatrix}$ and $\begin{bmatrix}u_{2} & u_{3}\end{bmatrix}$? <b>Yes</b>.

Then $u_{0}$ is a bad-bad channel, $u_{1}$ is a good-bad channel, $u_{2}$ is a bad-good channel, $u_{3}$ is a good-good channel.

The equation is below:

$$\begin{equation} \label{eq1}
\begin{split}
x & =\begin{bmatrix}u_{0} & u_{1} & u_{2} & u_{3}\end{bmatrix}=\begin{bmatrix}U_{01} & U_{23}\end{bmatrix}\\
w & =\begin{bmatrix}U_{01}G_{2} & U_{23}G_{2}\end{bmatrix}=\begin{bmatrix}w_{0} & w_{1} & w_{2} & w_{3}\end{bmatrix}\\
w' & =w\begin{bmatrix}1 & 0 & 0 & 0 \\ 0 & 0 & 1 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 1\end{bmatrix} = \begin{bmatrix}w'_{0} & w'_{1} & w'_{2} & w'_{3}\end{bmatrix} = \begin{bmatrix}W'_{01} & W'_{23}\end{bmatrix}\\
y' & = \begin{bmatrix}W'_{01}G_{2} & W'_{23}G_{2}\end{bmatrix} = \begin{bmatrix}y'_{0} & y'_{1} & y'_{2} & y'_{3}\end{bmatrix} \\
y & = \begin{bmatrix}y_{0} & y_{1} & y_{2} & y_{3}\end{bmatrix} = \begin{bmatrix}y'_{0} & y'_{2} & y'_{1} & y'_{3}\end{bmatrix} = y'\begin{bmatrix}1 & 0 & 0 & 0 \\ 0 & 0 & 1 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 1\end{bmatrix}
\end{split}
\end{equation}$$

One may note that $\begin{bmatrix}1 & 0 & 0 & 0 \\ 0 & 0 & 1 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 1\end{bmatrix}$ is a permutation matrix. All of the computations are linearly composed, thus, we may find a final form of this complex computation:

$$\begin{equation} \label{eq2}
\begin{split}
w & = x\begin{bmatrix}G_{2} & \mathbb{0} \\ \mathbb{0} & G_{2}\end{bmatrix} \\
w' & = w\begin{bmatrix}1 & 0 & 0 & 0 \\ 0 & 0 & 1 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 1\end{bmatrix} \\
y' & = w'\begin{bmatrix}G_{2} & \mathbb{0} \\ \mathbb{0} & G_{2}\end{bmatrix} \\
y & = y'\begin{bmatrix}1 & 0 & 0 & 0 \\ 0 & 0 & 1 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 1\end{bmatrix}
\end{split}
\end{equation}$$

Thus,

$$\begin{equation} \label{eq3}
\begin{split}
y & = x \begin{bmatrix}G_{2} & \mathbb{0} \\ \mathbb{0} & G_{2}\end{bmatrix} \begin{bmatrix}1 & 0 & 0 & 0 \\ 0 & 0 & 1 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 1\end{bmatrix} \begin{bmatrix}G_{2} & \mathbb{0} \\ \mathbb{0} & G_{2}\end{bmatrix} \begin{bmatrix}1 & 0 & 0 & 0 \\ 0 & 0 & 1 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 1\end{bmatrix} \\
& = x \begin{bmatrix}1 & 0 & 0 & 0 \\ 1 & 1 & 0 & 0 \\ 0 & 0 & 1 & 0 \\ 0 & 0 & 1 & 1\end{bmatrix} \begin{bmatrix}1 & 0 & 0 & 0 \\ 0 & 0 & 1 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 1\end{bmatrix} \begin{bmatrix}1 & 0 & 0 & 0 \\ 1 & 1 & 0 & 0 \\ 0 & 0 & 1 & 0 \\ 0 & 0 & 1 & 1\end{bmatrix} \begin{bmatrix}1 & 0 & 0 & 0 \\ 0 & 0 & 1 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 1\end{bmatrix}
\end{split}
\end{equation}$$

Then we can calculate the 4-bit polarization generator matrix as below:

In [19]:
G2_i = np.array([[1,0,0,0],
                [1,1,0,0],
                [0,0,1,0],
                [0,0,1,1]])
Perm = np.array([[1,0,0,0],
                [0,0,1,0],
                [0,1,0,0],
                [0,0,0,1]])

G4 = np.matmul(np.matmul(np.matmul(G2_i,Perm),G2_i),Perm)

print("G4:")
print(G4)

G4:
[[1 0 0 0]
 [1 1 0 0]
 [1 0 1 0]
 [1 1 1 1]]


Note that the $G_{4}$ just looks like:

$$G_{4} = \begin{bmatrix}1 & 0 & 0 & 0 \\ 1 & 1 & 0 & 0 \\ 1 & 0 & 1 & 0 \\ 1 & 1 & 1 & 1\end{bmatrix} = \begin{bmatrix}G_{2} & \mathbb{0} \\ G_{2} & G_{2}\end{bmatrix} = G_{2} \begin{bmatrix}\mathbb{I} & \mathbb{0} \\ \mathbb{I} & \mathbb{I}\end{bmatrix} = G_{2} \mathbb{G}_{2}$$

This matrix operation is called the <b><i>Kronecker product</i></b>. You may notice later that this Kronecker product can be shown in other coding theory papers such as OFDM <i>Hadamard matrix</i> construction, and so on.

Constructing $G_{8}$ is now simple as $G_{8} = G_{4} \mathbb{G}_{2}$. We may use recursive function for this.

Now we may implement such polarization coding (<i>i.e. <b>Polar Code</b></i>) as below. This encoding function was implemented as a distinguished <i>.py</i> file.